<a href="https://colab.research.google.com/github/zhiyangg/aps360/blob/main/APS360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("hello world")

hello world
